In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np

def generate_rest_url_extension(loc):
    restaurant_ext_list = []
    processing_time_list = []
    
    print(f'function generate_rest_url_extension ({loc}) started')
    site_url = "https://pergikuliner.com"
    url_page_extension = "/restoran/bandung/{}/?page={}"
    
    page = 1
    time_start = datetime.now().replace(microsecond=0)
    processing_time_start = datetime.now().replace(microsecond=0)
    while(True):
        processing_time_end = datetime.now().replace(microsecond=0)
        
        processing_time = processing_time_end - processing_time_start
        processing_time_list.append(processing_time)
        
        processing_time_start = datetime.now().replace(microsecond=0)
        
        if page%25 == 0:
            print(str(page) + ' done with ' + str(np.mean(processing_time_list)) + ' average process time')
            processing_time_list = []
            
        page_url = site_url + url_page_extension
        page_request = requests.get(page_url.format(loc, page))
        page_soup = BeautifulSoup(page_request.content, 'lxml')

        restaurant_content_div = page_soup.find('div', id='restaurant_contents')
        if restaurant_content_div:
            
            restaurant_figures = restaurant_content_div.find_all('figure')

            if restaurant_figures:
                for restaurant_figure in restaurant_figures:
                    restaurant_ext_list.append(restaurant_figure.find('a')['href'] + ' \n')
                    
            else:
                file = open(f"{loc}_rest_exten.txt", "w")
                file.writelines(restaurant_ext_list)
                file.close()

                time_end = datetime.now().replace(microsecond=0)

                print(f'function generate_rest_url_extension ({loc}) finished')
                print(f'execution time : {time_end - time_start}')
                print('')
                print('----------------------------------------------')
                print('')
                break
                    
        else:
            
            file = open(f"{loc}_rest_exten.txt", "w")
            file.writelines(restaurant_ext_list)
            file.close()
            
            time_end = datetime.now().replace(microsecond=0)
            
            print(f'function generate_rest_url_extension ({loc}) finished')
            print(f'execution time : {time_end - time_start}')
            print('')
            print('----------------------------------------------')
            print('')
            break
            
        page += 1
        
if __name__ == '__main__':
    generate_rest_url_extension('riau')
    generate_rest_url_extension('pasir-kaliki')
    generate_rest_url_extension('dago')
    generate_rest_url_extension('pasteur')
    generate_rest_url_extension('dipatiukur')

function generate_rest_url_extension (riau) started
25 done with 0:00:01.880000 average process time
50 done with 0:00:01.680000 average process time
75 done with 0:00:01.520000 average process time
100 done with 0:00:01.560000 average process time
125 done with 0:00:01.640000 average process time
function generate_rest_url_extension (riau) finished
execution time : 0:03:32

----------------------------------------------

function generate_rest_url_extension (pasir-kaliki) started
25 done with 0:00:01.520000 average process time
50 done with 0:00:01.480000 average process time
75 done with 0:00:01.360000 average process time
function generate_rest_url_extension (pasir-kaliki) finished
execution time : 0:02:20

----------------------------------------------

function generate_rest_url_extension (dago) started
25 done with 0:00:01.640000 average process time
50 done with 0:00:01.440000 average process time
75 done with 0:00:01.560000 average process time
100 done with 0:00:01.360000 aver

In [2]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import re
import pandas as pd

def crawl_restaurant_info(filename):
    file = open(filename, "r")
    rest_url_exten_list = file.read().split(' \n')
    return rest_url_exten_list

In [3]:
def get_restaurant_info(url_exten, time_crawled):
    site_url = "https://pergikuliner.com"
    
    page_url = site_url + url_exten
    page_request = requests.get(page_url)
    page_soup = BeautifulSoup(page_request.content, 'lxml')

    resto_id = time_crawled.replace('.','')
    # gather restaurant info
    
    # get location
    loc_idx = str(page_soup).find('"lat"')
    loc_text = str(page_soup)[loc_idx:loc_idx+50]
    loc_text_regex = r'[^a-zA-Z\s\"\:\,]+'
    lat, long = re.findall(loc_text_regex, loc_text)
    lat, long = float(lat), float(long)

    height_mark_soup = page_soup.find('div', id='height-mark')
    heading_soup = height_mark_soup.find('div', class_='heading')
    restaurant_name = heading_soup.find('h1').text
    cuisine_type = heading_soup.find('span', class_='cuisine-type').text
    is_official = bool(heading_soup.find('img')) #img represents the verified logo
    location_text = re.sub(r'(\s)+', ' ', height_mark_soup.find('span', class_='left').text)
    average_price = re.sub(r'(\s)+', ' ', height_mark_soup.find('span', id='avg-price').text)
    overall_rating_soup = height_mark_soup.find('span', itemprop='ratingValue')
    overall_rating = float(overall_rating_soup.text) if overall_rating_soup else 0.0
    detail_ratings_soup = height_mark_soup.find('div', class_='detail-review')
    detail_ratings = []

    # detail_ratings = [('rasa', rasa_rating), ('suasana', suasana_rating), ...]
    if detail_ratings_soup:
        for detail_rating_soup in detail_ratings_soup.find_all('div', recursive=False):
            details = detail_rating_soup.find_all('div')
            detail_ratings.append(float(details[1].text))
    else:
        detail_ratings = [0] * 5

    content_detail_soup = page_soup.find('div', id='content-detail')
    open_time = 'not listed'
    if content_detail_soup.find('time'):
        open_time = content_detail_soup.find('time').text
    
    payment_method = 'not listed'
    if content_detail_soup.find('meta'):
        payment_method = content_detail_soup.find('meta')['content']

    facility_soup = content_detail_soup.find('div', class_='facility-list')
    facilities = []
    if facility_soup:
        for facility in facility_soup.find_all('label'):
            if facility['class']:
                facilities.append('yes' if facility['class'][0] == 'checked' else 'no')
            else:
                facilities.append("not sure")
    else:
        facilities = ['not listed'] * 11
        
    return [resto_id, restaurant_name, cuisine_type, is_official, lat, long, location_text,
           average_price, overall_rating] + detail_ratings + [open_time, payment_method] +\
           facilities

def get_restaurant_reviews(url_exten, time_crawled):
    site_url = "https://pergikuliner.com"
    
    page = 1
    page_url_format = site_url + url_exten + '?page={}'
    
    error_user_id = []
    prev_error_user_id = ['a']
    conseq_error_count = 0
    review_list = []
    user_info_list = []
    
    while(True):
        error_user_id = []
        page_url = page_url_format.format(str(page))
        page_request = requests.get(page_url)
        page_soup = BeautifulSoup(page_request.content, 'lxml')
        review_soup_list = page_soup.find_all('div', class_='review-wrapper')
        extra_spaces_regex = r'(\s)+'
        if error_user_id:
            prev_error_user_id = error_user_id
        
        if review_soup_list and page<30:
            for review_soup in review_soup_list:
                try:
                    # user info
                    user_info_soup = review_soup.find('div', class_='user')
                    user_id = user_info_soup.find('a')['href'].split('/')[1]
                    username = re.sub(extra_spaces_regex, ' ', user_info_soup.find('span', itemprop='name').text).strip()
                    alpha_badge = 'no badge'
                    if user_info_soup.find('p', class_='alpha-badge'):
                        alpha_badge = user_info_soup.find('p', class_='alpha-badge').text
                    user_info_details = user_info_soup.find('div', class_='usr-info').find('div').find_all('p')[-2:]
                    review_count = int(user_info_details[0].text.split()[0])
                    account_level = int(user_info_details[1].text.split()[1])

                    badge_container_soup = user_info_soup.find('p', class_='badge-container')
                    badge_list = badge_container_soup.find_all('img')
                    badges = 'no badge'
                    if badge_list:
                        badges = ''
                        for badge in badge_list:
                            badges += badge['alt'] + ','

                    user_info = [user_id, username, alpha_badge, review_count, account_level, badges]
                    user_info_list.append(user_info)
                    
                    # review details
                    review_detail_soup = review_soup.find('div', class_='content')
                    review_title = review_detail_soup.find('a', class_='link-more-review').text
                    overall_rating = review_detail_soup.find('span', class_='item-rating-result')
                    overall_rating = float(overall_rating.find('span').find('span').text.split()[0])
                    review_text = review_detail_soup.find('div', itemprop='reviewBody').find('p').text
                    review_date = review_detail_soup.find('meta', itemprop='datePublished')['content']
                    review_helpful_count = 0
                    if review_detail_soup.find('div', class_='helpful-counts'):
                        review_helpful_count = review_detail_soup.find('div', class_='helpful-counts').text.split()[0]
                    review_data = [user_id, time_crawled, review_title, overall_rating, review_date, 
                                   review_helpful_count, review_text]
                    review_list.append(review_data)
                except:
                    error_user_id.append(user_id)
        else:
            break
            
        # unknown error handling
        if (prev_error_user_id == error_user_id) and error_user_id:
            conseq_error_count += 1
        if conseq_error_count > 2:
            break
        prev_error_user_id = error_user_id
        page += 1
    return user_info_list, review_list

def crawl_pergiKuliner(filename):
    
    processing_time_list = []
    restaurant_info_col_names = [
        'resto_id', 'name', 'cuisine_type', 'is_official', 'lat', 'long', 'location',
        'average_price', 'overall_rating', 'rating_rasa', 'rating_suasana', 'rating_harga_rasa',
        'rating_pelayanan', 'rating_kebersihan', 'open_time', 'payment_method', 'wifi', 
        'delivery', 'smoking_area', 'outdor_area', '24hour', '100%_halal', 'VIP_room',
        'reservation', 'parking_area', 'alcohol', 'vegetarian'
    ]
    restaurant_info_df = pd.DataFrame(columns = restaurant_info_col_names)
    
    user_col_names = ['user_id', 'username', 'alpha_badge', 'review_count',
                  'account_level', 'badges']
    user_info_df = pd.DataFrame(columns = user_col_names)
    
    review_col_names = ['user_id', 'resto_id', 'review_title', 'overall_rating', 'review_date', 
                        'review_helpful_count', 'review_text',
                                   ]
    review_df = pd.DataFrame(columns = review_col_names)
    
    with open(filename) as f:
        url_extensions = f.readlines()
        
    n = 1
    for url_extension in url_extensions:
        if n%100 == 0:
            print(str(n) + ' done with ' + str(np.mean(processing_time_list)) + ' average process time')
            processing_time_list = []
            
        print(url_extension)
            
        processing_time_start = datetime.now().replace(microsecond=0)
        
        url_extension = re.sub(r'(\s)', '', url_extension)
        if url_extension:
            time_crawled = str(datetime.now().timestamp())
            
            # restaurant_info
            restaurant_info_data = get_restaurant_info(url_extension, time_crawled)
            new_row = dict(zip(restaurant_info_col_names, restaurant_info_data))
            restaurant_info_df = restaurant_info_df.append(new_row, ignore_index=True)
            
            user_info_data, restaurant_review_data = get_restaurant_reviews(url_extension, time_crawled)
            # user_info
            for user_info in user_info_data:    
                new_row = dict(zip(user_col_names, user_info))
                user_info_df = user_info_df.append(new_row, ignore_index=True)
                
            # review details
            for review_data in restaurant_review_data:
                new_row = dict(zip(review_col_names, review_data))
                review_df = review_df.append(new_row, ignore_index=True)
            
        processing_time_end = datetime.now().replace(microsecond=0)
        
        processing_time = processing_time_end - processing_time_start
        processing_time_list.append(processing_time)
        n += 1
    filename_for_restaurant_info = filename.split('_')[0] + '-restaurant-info.csv'
    filename_for_user_info = filename.split('_')[0] + '-user-info.csv'
    filename_for_review_data = filename.split('_')[0] + '-review-data.csv'
    restaurant_info_df.to_csv(filename_for_restaurant_info)
    user_info_df.to_csv(filename_for_user_info)
    review_df.to_csv(filename_for_review_data)

In [9]:
# crawl_pergiKuliner('riau_rest_exten.txt')
# crawl_pergiKuliner('pasir-kaliki_rest_exten.txt')
# crawl_pergiKuliner('dago_rest_exten.txt')
# crawl_pergiKuliner('pasteur_rest_exten.txt')
# crawl_pergiKuliner('dipatiukur_rest_exten.txt')

/restaurants/bandung/starbucks-coffee-dipatiukur 

/restaurants/bandung/lucio-luciano-pattiseries-dipatiukur 

/restaurants/bandung/de-u-coffee-dipatiukur 

/restaurants/bandung/delapan-padi-dipatiukur 

/restaurants/bandung/miluyu-coffee-lounge-dipatiukur 

/restaurants/bandung/warung-nasi-spg-dipatiukur-2 

/restaurants/bandung/psy-steamboat-yakiniku-dipatiukur 

/restaurants/bandung/kumari-dipatiukur 

/restaurants/bandung/seduh-sembuh-dipatiukur 

/restaurants/bandung/kawan-dipatiukur 

/restaurants/bandung/sambal-lalap-wong-solo-dipatiukur 

/restaurants/bandung/becky-s-brunch-dan-dine-dipatiukur 

/restaurants/bandung/infinito-disconnect-dipatiukur 

/restaurants/bandung/sei-sapi-lamalera-dipatiukur 

/restaurants/bandung/bakmi-jowo-du67-dipatiukur 

/restaurants/bandung/ayam-geprek-bebas-dipatiukur-2 

/restaurants/bandung/baso-aci-ganteng-dipatiukur 

/restaurants/bandung/betah-space-dipatiukur 

/restaurants/bandung/nasi-goreng-simongol-dipatiukur 

/restaurants/bandung/nasi-g

/restaurants/bandung/sate-dan-soto-ayam-kak-irfan-dipatiukur 

/restaurants/bandung/kopmil-street-coffee-dipatiukur 

/restaurants/bandung/ayam-penyet-bu-erte-dipatiukur-2 

/restaurants/bandung/skoteng-dan-susu-murni-eyang-subur-dipatiukur 

/restaurants/bandung/drink-217-dipatiukur 

/restaurants/bandung/kawecumi-dipatiukur 

/restaurants/bandung/sate-padang-ajo-pariaman-dipatiukur-2 

/restaurants/bandung/ayam-bakar-padang-dipatiukur 

/restaurants/bandung/ayam-gepuk-pak-gembus-dipatiukur 

/restaurants/bandung/mirasa-dipatiukur 

/restaurants/bandung/warung-nasi-ibu-euis-dipatiukur 

/restaurants/bandung/warung-kalimbo-dipatiukur 

/restaurants/bandung/kedai-baso-mas-mino-dipatiukur 

/restaurants/bandung/pempek-lugina-dipatiukur 

/restaurants/bandung/bakso-ikan-xpress-spesial-dipatiukur 

/restaurants/bandung/cindy-fresh-juice-dipatiukur 

/restaurants/bandung/sosis-serdadu-dipatiukur 

/restaurants/bandung/basmara-eatery-dipatiukur 

/restaurants/bandung/susu-murni-geboy-dipatiu

/restaurants/bandung/sabana-fried-chicken-dipatiukur-2 

/restaurants/bandung/kantin-teh-unik-om-dipatiukur 

/restaurants/bandung/kopi-truck-mutiara-papandayan-dipatiukur 

/restaurants/bandung/glamor-chiken-dipatiukur 

/restaurants/bandung/juice-sehat-d-dan-g-dipatiukur 

/restaurants/bandung/nasi-rawon-parjo-dipatiukur 

/restaurants/bandung/susu-murni-ajip-dipatiukur 

/restaurants/bandung/nana-cappucino-dipatiukur 

/restaurants/bandung/nasi-kremes-sunda-dipatiukur 

/restaurants/bandung/krenchise-hot-spicy-chicken-dipatiukur 

/restaurants/bandung/starblend-coffee-dan-donuts-dipatiukur 

/restaurants/bandung/raqil-juice-dipatiukur 

/restaurants/bandung/b-gana-dipatiukur 

/restaurants/bandung/batagor-robi-dipatiukur 

/restaurants/bandung/grand-kebab-dipatiukur 

/restaurants/bandung/ciphai-dipatiukur 

/restaurants/bandung/plo-penangkis-lapar-orang-dipatiukur 

/restaurants/bandung/rafa-juice-dipatiukur 

/restaurants/bandung/d-besto-dipatiukur 

/restaurants/bandung/rizka-mor

/restaurants/bandung/bandung-makuta-dipatiukur 

/restaurants/bandung/steak-n-snack-dipatiukur 

/restaurants/bandung/warung-tenda-biru-dipatiukur 

/restaurants/bandung/kini-cheese-tea-dipatiukur 

/restaurants/bandung/coffice-by-coffindo-dipatiukur 

/restaurants/bandung/king-dessert-dipatiukur 

/restaurants/bandung/dim-sum-lab-dipatiukur 

/restaurants/bandung/kopi-om-dipatiukur 

/restaurants/bandung/warkop-add-dipatiukur 

/restaurants/bandung/liwet-jambal-88-dipatiukur 

/restaurants/bandung/d-r-e-i-street-coffee-dipatiukur 

/restaurants/bandung/sangu-rice-bowl-dipatiukur 

/restaurants/bandung/bananugget-alading-dipatiukur 

/restaurants/bandung/temperature-coffee-dipatiukur 

/restaurants/bandung/mocking-mocktails-dipatiukur 

/restaurants/bandung/ayam-geprek-wisuda-dipatiukur 

/restaurants/bandung/waroeng-ikan-bakar-sambal-pesisir-dipatiukur 

/restaurants/bandung/kedai-kopi-bbm-dipatiukur 

/restaurants/bandung/low-cost-seafood-dipatiukur 

500 done with 0:00:01.020000 ave

/restaurants/bandung/mie-tomyum-nin-nen-dipatiukur 

/restaurants/bandung/super-geprek-dipatiukur 

/restaurants/bandung/pucok-kang-nashir-dipatiukur 

/restaurants/bandung/baso-tahu-hokkie-dipatiukur 

/restaurants/bandung/kantin-du-dipatiukur 

/restaurants/bandung/soto-spesial-pangandaran-hj-iin-dipatiukur 

/restaurants/bandung/waroeng-dimsum-dipatiukur 

/restaurants/bandung/dimsum-168-dipatiukur 

/restaurants/bandung/chick-n-rich-dipatiukur 

/restaurants/bandung/mister-seafood-dipatiukur 

/restaurants/bandung/batagor-saskia-dipatiukur 

/restaurants/bandung/batagor-hanimun-dipatiukur 

/restaurants/bandung/keep-coffee-dipatiukur 

/restaurants/bandung/kochi-coffee-dipatiukur 

/restaurants/bandung/rocket-dimsum-dipatiukur 

/restaurants/bandung/kopilak-dipatiukur-2 

/restaurants/bandung/mie-bakso-mak-doeloe-dipatiukur 

/restaurants/bandung/warung-nasi-mak-doeloe-dipatiukur 

/restaurants/bandung/shinju-ramen-dipatiukur 

/restaurants/bandung/milkchester-dipatiukur 

/restaur

/restaurants/bandung/soto-sulung-mas-madu-dipatiukur 

/restaurants/bandung/smip-yppt-93-dipatiukur 

/restaurants/bandung/lambe-nyinyir-dipatiukur 

/restaurants/bandung/d-fie-juice-dipatiukur 

/restaurants/bandung/warung-junet-dipatiukur 

/restaurants/bandung/summo-dipatiukur 

/restaurants/bandung/soto-betawi-asli-abah-entis-dipatiukur 

/restaurants/bandung/dimsum-dan-noodle-nell-q-dipatiukur 

/restaurants/bandung/picco-dipatiukur 

/restaurants/bandung/d-trin-dipatiukur 

/restaurants/bandung/kapau-si-bungsu-dipatiukur 

/restaurants/bandung/dapur-minang-dipatiukur 

/restaurants/bandung/focus-corner-dipatiukur 

/restaurants/bandung/dapur-kencana-dipatiukur 

/restaurants/bandung/warung-nasi-sunda-priangan-jaya-dipatiukur 

/restaurants/bandung/ngopi-doeloe-dipatiukur 

/restaurants/bandung/baso-ceu-hetty-dipatiukur 

/restaurants/bandung/seblak-tom-tom-dipatiukur 

/restaurants/bandung/double-eight-coffee-dipatiukur 

/restaurants/bandung/mi-ku-bagusrangin-dipatiukur 

/resta

In [5]:
page_url_format = 'https://pergikuliner.com/restaurants/jakarta/furusato-izakaya-sudirman?page={}'

In [17]:
# page = 1
# page_url = page_url_format.format(str(page))
# page_request = requests.get(page_url)
page_soup = BeautifulSoup(page_request.content, 'lxml')
review_soup_list = page_soup.find_all('div', class_='review-wrapper')

In [37]:
# if review_soup_list:
review_detail_soup = review_soup_list[1].find('div', class_='content')
review_title = review_detail_soup.find('a', class_='link-more-review').text
overall_rating = review_detail_soup.find('span', class_='item-rating-result')
overall_rating = float(overall_rating.find('span').find('span').text.split()[0])
review_text = review_detail_soup.find('div', itemprop='reviewBody').find('p').text
# .find('div', class_='bottom-part')
review_date = review_detail_soup.find('meta', itemprop='datePublished')['content']
review_helpful_count = 0
if review_detail_soup.find('div', class_='helpful-counts'):
    review_helpful_count = review_detail_soup.find('div', class_='helpful-counts').text.split()[0]

In [36]:
review_detail_soup.find('div', class_='helpful-counts').text.split()[0]

'2'

In [24]:
float(review_detail_soup.find('span', class_='item-rating-result').find('span').find('span').text.split()[0])

'4.6'